## Downloading 3DEP Lidar data

1. Visit https://apps.nationalmap.gov/lidar-explorer/#/
2. Select the area you would like to download
3. On the right pane, under Downloadable Products Within AOI/Lidar within AOI, click the button to the right of Lidar Point Cloud (LPC) to get the download list
4. Put the downloadlist.txt file into the root directory of this project

## Bash commands to download the data from the download list


In [ ]:
!rm -rf data/
!mkdir data/
!wget -i downloadlist.txt -P data/

## Read the lidar data files
This step reads the downloaded .laz files in data/, converts the lidar samples to standard gps lat/long/elevation coordinates, and concatenates all the samples into one large array.

In [1]:
import os
import math

import laspy
from pyproj import CRS
from pyproj import Transformer

import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib widget

In [2]:
data_path = 'data/'

file_names = os.listdir(data_path)

total_point_count = 0

for file_name in file_names:    
    # open the file with laspy
    las_file = laspy.open(data_path + file_name)
    total_point_count = total_point_count + las_file.header.point_count

    # print(f'File {data_path + file_name} contains {las_file.header.point_count} points')

print(f'Files contain a total of {total_point_count} points which amounts to {8*3*(total_point_count/1024)/1024} MB')


Files contain a total of 6879619 points which amounts to 157.46198272705078 MB


In [3]:
lidar_points_raw = np.zeros((total_point_count, 3), dtype='float64')
start_idx = 0

for file_name in file_names:
    print(f'File {data_path + file_name}:')
    print(f'    reading file')

    # open the file with laspy
    las_file = laspy.read(data_path + file_name)

    print(f'    transforming and concatinating points')
    # read the spatial reference information    
    crs = CRS.from_wkt(las_file.vlrs.get("WktCoordinateSystemVlr")[0].string)

    # create a transformer to transform to EPSG:4978, a geocentric cartesian 3d coordinate reference system
    transformer = Transformer.from_crs(crs, '4978')

    # convert the raw lidar points to gps lat/long
    transform_iter = (pt for pt in transformer.itransform(las_file.xyz))
    lidar_points_raw[start_idx:(start_idx + las_file.header.point_count), :] = np.fromiter(transform_iter, dtype=((las_file.xyz.dtype, 3)))

    start_idx = start_idx + las_file.header.point_count

print("Done")

File data/USGS_LPC_GA_Statewide_2018_B18_DRRA_e1072n1350.laz:
    reading file
    transforming and concatinating points
Done


In [7]:
# decimation = 1

# lidar_points = np.copy(lidar_points_raw[0::decimation, :])

# # set coordinate reference to center
# avg_lidar_points = np.mean(lidar_points)
# lidar_points = lidar_points - avg_lidar_points


# # creating figure
# plt.ion()
# fig = plt.figure()
# ax = fig.add_subplot(projection='3d')

# # creating the plot
# ax.scatter(lidar_points[:, 1], lidar_points[:, 0], lidar_points[:, 2], s=1.0, c=lidar_points[:, 2])
# ax.set_title("Lidar data")
# ax.set_xlabel("Longitute")
# ax.set_ylabel("Lattitude")
# plt.show()


## Generate a 3d surface from the lidar point cloud

In [4]:
import open3d as o3d

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(lidar_points_raw)
cl, ind = pcd.remove_statistical_outlier(nb_neighbors=20, std_ratio=2.0)
o3d.visualization.draw_geometries([pcd])

# estimate point normals
# pcd.estimate_normals()

# distances = pcd.compute_nearest_neighbor_distance()
# avg_dist = np.mean(distances)
# radius = 1.5 * avg_dist   

# mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(
#            pcd, o3d.utility.DoubleVector([radius, radius * 2]))

# with o3d.utility.VerbosityContextManager(o3d.utility.VerbosityLevel.Debug) as cm:
#     mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd)

# mesh.compute_vertex_normals()
# mesh.paint_uniform_color([1, 0.706, 0])
# o3d.visualization.draw_geometries([mesh])



Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
